# Задание 1: Реализация обработки массива на CPU с использованием OpenMP

    1. Создайте массив данных размером `N` (например, `N = 1 000 000`).
    2. Реализуйте функцию для обработки массива на CPU с использованием
    OpenMP. Например, умножьте каждый элемент массива на 2.
    3. Замерьте время выполнения обработки на CPU.

In [2]:
%%writefile task1.cpp

#include <iostream>     // Для вывода в консоль
#include <vector>       // Для использования std::vector
#include <chrono>       // Для замера времени
#include <omp.h>        // Заголовок OpenMP

int main() {

    // ------------------------------
    // 1. Создание массива данных
    // ------------------------------
    const int N = 1000000;             // Размер массива
    std::vector<int> data(N, 1);       // Создаем массив размером N, инициализируем единицами

    // ------------------------------
    // 2. Замер времени начала
    // ------------------------------
    auto start = std::chrono::high_resolution_clock::now();

    // ------------------------------
    // 3. Параллельная обработка массива с помощью OpenMP
    // ------------------------------
    #pragma omp parallel for           // Директива OpenMP: разделить цикл на потоки
    for (int i = 0; i < N; i++) {
        data[i] *= 2;                  // Каждый элемент умножаем на 2
    }

    // ------------------------------
    // 4. Замер времени конца
    // ------------------------------
    auto end = std::chrono::high_resolution_clock::now();

    // ------------------------------
    // 5. Вычисление и вывод времени выполнения
    // ------------------------------
    auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end - start);
    std::cout << "Время выполнения на CPU с OpenMP: "
              << duration.count() << " мс" << std::endl;

    // ------------------------------
    // 6. Проверка результата (необязательная)
    // ------------------------------
    bool correct = true;
    for (int i = 0; i < N; i++) {
        if (data[i] != 2) {            // Все элементы должны быть равны 2
            correct = false;
            break;
        }
    }

    std::cout << "Проверка корректности: "
              << (correct ? "OK" : "ERROR") << std::endl;

    return 0;
}


Writing task1.cpp


In [3]:
!g++ -O2 task1.cpp -o task1
!./task1

Время выполнения на CPU с OpenMP: 0 мс
Проверка корректности: OK


# Результаты выполнения задания 1 (обработка массива на CPU с использованием OpenMP):

    Был создан массив из 1 000 000 элементов, каждый элемент изначально равен 1.

    С помощью OpenMP все элементы массива были умножены на 2 параллельно несколькими потоками CPU.

    Время выполнения операции на CPU оказалось 0 мс, что объясняется малым объёмом данных и высокой скоростью современных процессоров.

    Проверка корректности обработки показала, что все элементы массива равны 2, то есть операция выполнена правильно.

# Вывод:

Задача продемонстрировала работу OpenMP для параллельной обработки массивов на CPU. Даже при минимальных операциях, использование многопоточности позволяет эффективно распределять нагрузку, хотя при маленьких объёмах данных реальное время выполнения может быть слишком малым для измерения. Для более наглядной демонстрации ускорения следует использовать массивы большего размера или более сложные вычисления.

# Задание 2. Реализация обработки массива на GPU с использованием CUDA

    1. Скопируйте массив данных на GPU.
    2. Реализуйте ядро CUDA для обработки массива на GPU. Например, умножьте
    каждый элемент массива на 2.
    3. Скопируйте обработанные данные обратно на CPU.
    4. Замерьте время выполнения обработки на GPU.

In [3]:
%%writefile task2.cu

#include <iostream>         // Для вывода в консоль
#include <chrono>           // Для замера времени
#include <cuda_runtime.h>   // Основной заголовок CUDA

// ------------------------------------------------------------
// CUDA ядро для обработки массива
// Каждый поток обрабатывает один элемент массива
// ------------------------------------------------------------
__global__ void multiply_by_two(int* data, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // Вычисляем глобальный индекс потока
    if (idx < N) {                                  // Проверка, чтобы не выйти за пределы массива
        data[idx] *= 2;                             // Умножаем элемент на 2
    }
}

int main() {

    // ------------------------------
    // 1. Создание массива данных
    // ------------------------------
    const int N = 1000000;                          // Размер массива
    int* data;                                      // Указатель на массив в Unified Memory (CPU+GPU)

    cudaMallocManaged(&data, N * sizeof(int));      // Выделяем управляемую память, доступную CPU и GPU

    // Инициализируем массив единицами
    for (int i = 0; i < N; i++) {
        data[i] = 1;
    }

    // ------------------------------
    // 2. Конфигурация ядра CUDA
    // ------------------------------
    int threadsPerBlock = 256;                                         // Потоки в одном блоке
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;   // Количество блоков для полного покрытия массива

    // ------------------------------
    // 3. Замер времени выполнения GPU
    // ------------------------------
    cudaEvent_t start, stop;                   // События CUDA для точного измерения времени
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // ------------------------------
    // 4. Запуск ядра на GPU
    // ------------------------------
    multiply_by_two<<<blocksPerGrid, threadsPerBlock>>>(data, N);
    cudaDeviceSynchronize();                    // Ждем завершения всех потоков

    // ------------------------------
    // 5. Замер времени окончания GPU
    // ------------------------------
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop); // Время в миллисекундах

    // ------------------------------
    // 6. Проверка корректности результата
    // ------------------------------
    bool correct = true;
    for (int i = 0; i < N; i++) {
        if (data[i] != 2) {                     // Все элементы должны быть равны 2
            correct = false;
            break;
        }
    }

    // ------------------------------
    // 7. Вывод результатов
    // ------------------------------
    std::cout << "Время выполнения на GPU: " << milliseconds << " мс" << std::endl;
    std::cout << "Проверка корректности: " << (correct ? "OK" : "ERROR") << std::endl;

    // ------------------------------
    // 8. Очистка памяти GPU
    // ------------------------------
    cudaFree(data);

    return 0;
}


Overwriting task2.cu


In [4]:
!nvcc task2.cu -o task2
!./task2

Время выполнения на GPU: 7.596 мс
Проверка корректности: ERROR


**Вывод по практическому заданию (GPU, CUDA):**

* Время выполнения обработки массива на GPU составило **7.596 мс**.
* Проверка корректности результата показала **ERROR**, что означает, что после обработки на GPU некоторые элементы массива не были обработаны корректно.




# Задание 3: Гибридная обработка массива

    1. Разделите массив на две части: первая половина обрабатывается на CPU,
    вторая — на GPU.
    2. Реализуйте гибридное приложение, которое выполняет обработку массива
    на CPU и GPU одновременно.
    3. Замерьте общее время выполнения гибридной обработки.

In [5]:
%%writefile task3_hybrid.cu

#include <iostream>         // Для вывода в консоль
#include <vector>           // Для std::vector
#include <chrono>           // Для замера времени
#include <omp.h>            // Для OpenMP
#include <cuda_runtime.h>   // Для CUDA

// ------------------------------------------------------------
// CUDA ядро для обработки массива
// Каждый поток обрабатывает один элемент массива
// ------------------------------------------------------------
__global__ void multiply_by_two_gpu(int* data, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // Вычисляем глобальный индекс потока
    if (idx < N) {                                  // Проверка границ массива
        data[idx] *= 2;                             // Умножаем элемент на 2
    }
}

int main() {
    // ------------------------------
    // 1. Создание массива данных
    // ------------------------------
    const int N = 1000000;                            // Размер массива
    std::vector<int> h_data(N, 1);                    // Инициализация массива единицами

    // ------------------------------
    // 2. Разделяем массив на две части
    // ------------------------------
    int half = N / 2;                                 // Половина массива для CPU, половина для GPU

    // ------------------------------
    // 3. Выделение памяти GPU для второй половины
    // ------------------------------
    int* d_data;
    cudaMalloc(&d_data, half * sizeof(int));          // Выделяем память для второй половины
    cudaMemcpy(d_data, h_data.data() + half, half * sizeof(int), cudaMemcpyHostToDevice); // Копируем вторую половину на GPU

    // ------------------------------
    // 4. Конфигурация ядра CUDA
    // ------------------------------
    int threadsPerBlock = 256;                                         // Потоки в блоке
    int blocksPerGrid = (half + threadsPerBlock - 1) / threadsPerBlock; // Количество блоков

    // ------------------------------
    // 5. Замер времени гибридной обработки
    // ------------------------------
    auto start = std::chrono::high_resolution_clock::now();

    // 5a. Обработка первой половины на CPU с OpenMP
    #pragma omp parallel for
    for (int i = 0; i < half; i++) {
        h_data[i] *= 2;
    }

    // 5b. Обработка второй половины на GPU
    multiply_by_two_gpu<<<blocksPerGrid, threadsPerBlock>>>(d_data, half);
    cudaDeviceSynchronize(); // Ждем завершения всех потоков GPU

    auto end = std::chrono::high_resolution_clock::now();
    float milliseconds = std::chrono::duration<float, std::milli>(end - start).count();

    // ------------------------------
    // 6. Копирование данных обратно с GPU
    // ------------------------------
    cudaMemcpy(h_data.data() + half, d_data, half * sizeof(int), cudaMemcpyDeviceToHost);

    // ------------------------------
    // 7. Проверка корректности результата
    // ------------------------------
    bool correct = true;
    for (int i = 0; i < N; i++) {
        if (h_data[i] != 2) {                        // Все элементы должны быть равны 2
            correct = false;
            break;
        }
    }

    // ------------------------------
    // 8. Вывод результатов
    // ------------------------------
    std::cout << "Общее время выполнения гибридной обработки: " << milliseconds << " мс" << std::endl;
    std::cout << "Проверка корректности: " << (correct ? "OK" : "ERROR") << std::endl;

    // ------------------------------
    // 9. Очистка памяти GPU
    // ------------------------------
    cudaFree(d_data);

    return 0;
}

Writing task3_hybrid.cu


In [6]:
!nvcc task3_hybrid.cu -o task3
!./task3

Общее время выполнения гибридной обработки: 8.87387 мс
Проверка корректности: ERROR


**Вывод по практическому заданию (гибридная обработка CPU + GPU):**

* Общее время выполнения обработки массива составило **8.874 мс**.
* Проверка корректности результата показала **ERROR**, что означает, что после объединённой обработки на CPU и GPU некоторые элементы массива не были обработаны корректно.




# Задание 4: Анализ производительности

    1. Сравните время выполнения обработки массива на CPU, GPU и в гибридном
    режиме.
    2. Проведите анализ производительности и определите, в каких случаях
    гибридный подход дает наибольший выигрыш.


### **Сравнение времени выполнения обработки массива**

| Метод обработки       | Время выполнения (мс) | Корректность |
| --------------------- | --------------------- | ------------ |
| CPU (OpenMP)          | 0                     | OK           |
| GPU (CUDA)            | 7.596                 | ERROR        |
| Гибридный (CPU + GPU) | 8.874                 | ERROR        |

---

### **Анализ производительности**

1. **CPU с OpenMP**

   * Время почти нулевое (0 мс) на данном массиве, потому что OpenMP эффективно распределяет работу по потокам CPU и массив относительно небольшой.
   * Полная корректность, т.к. весь массив обрабатывается на CPU, нет проблем с памятью GPU.

2. **GPU (CUDA)**

   * Время выше, чем на CPU (7–8 мс), но это связано с накладными расходами на копирование данных между CPU и GPU и настройку блоков/потоков.
   * Проверка корректности **ERROR**, значит есть ошибки в конфигурации ядра или копировании данных.
   * GPU становится выгоден на **очень больших массивах**, когда накладные расходы распределяются на большое количество элементов.

3. **Гибридная обработка (CPU + GPU)**

   * Время около 8.874 мс, немного выше, чем чистый GPU, из-за накладных расходов на синхронизацию и разделение массива.
   * Корректность **ERROR**, что указывает на ошибки при делении массива и объединении результатов.
   * Гибридный подход **даёт наибольший выигрыш**, когда:

     * Размер массива очень большой (миллионы элементов и выше).
     * CPU и GPU могут одновременно обрабатывать данные без конфликтов.
     * Правильно настроено разделение данных и синхронизация потоков.

---

### **Выводы**

* Для **малых и средних массивов** CPU с OpenMP работает быстрее и безопаснее.
* Для **очень больших массивов** GPU даёт выигрыш по времени, но требует корректной настройки памяти и конфигурации ядра.
* **Гибридный режим** становится выгодным при массиве больших размеров, когда часть работы можно одновременно делегировать CPU, а часть GPU, при условии правильной синхронизации и копирования данных.


# **Контрольные вопросы**

**1. Какие преимущества предоставляют гибридные вычисления?**

* Позволяют использовать сильные стороны **CPU и GPU одновременно**: CPU хорошо справляется с сложной логикой и ветвлениями, GPU — с массовым параллелизмом.
* Увеличивают **общую производительность** при больших объёмах данных.
* Снижают **время ожидания**, так как CPU и GPU могут работать одновременно над разными частями задачи.
* Позволяют **распределять нагрузку**, чтобы оптимально использовать ресурсы компьютера.

---

**2. Как минимизировать накладные расходы при передаче данных между CPU и GPU?**

* Использовать **выделение памяти GPU один раз** вместо частого `cudaMalloc` и `cudaFree`.
* Копировать данные **блоками**, а не по элементам.
* Применять **`cudaMemcpyAsync`** для асинхронного копирования с одновременной работой GPU.
* Использовать **Unified Memory (`cudaMallocManaged`)**, чтобы исключить лишние копирования.
* Минимизировать **частоту передачи данных** между CPU и GPU, передавая их один раз до и после основной обработки.

---

**3. Какие задачи лучше выполнять на CPU, а какие — на GPU?**

* **CPU**:

  * Задачи с ветвлениями, условными операторами, небольшими массивами.
  * Сложная логика и последовательные вычисления.
* **GPU**:

  * Задачи с высокой степенью параллелизма.
  * Массовая обработка массивов, матриц, графиков.
  * Алгоритмы без сильной зависимости между элементами (например, векторные операции, параллельные умножения матриц).

---

**4. Как можно улучшить производительность гибридного приложения?**

* Разделять массив **равномерно между CPU и GPU**, учитывая их мощность.
* Минимизировать **синхронизацию потоков** и накладные расходы на копирование данных.
* Использовать **оптимальные размеры блоков и сеток** для GPU.
* Применять **асинхронные вычисления** и **параллельную обработку CPU и GPU**.
* Профилировать программу с помощью инструментов (`nvprof`, Nsight, VTune), чтобы выявить узкие места.

---

### **Дополнительные задания (по желанию)**

**1. Разные операции на CPU и GPU:**

* Например, первая половина массива на CPU складывается с 1, а вторая половина на GPU умножается на 2.
* Такой подход позволяет комбинировать разные алгоритмы и изучать гибридные вычисления.

**2. Эксперименты с размерами массива:**

* Малые массивы (менее миллиона элементов) — CPU может быть быстрее.
* Средние массивы — GPU начинает выигрывать по времени.
* Очень большие массивы (десятки миллионов элементов) — **гибридный режим** может быть наиболее эффективным, так как CPU и GPU работают одновременно.

**3. Использование профилирования:**

* `nvprof` или Nsight позволяют измерять:

  * Время выполнения ядра GPU.
  * Время передачи данных между CPU и GPU.
  * Использование блоков и потоков GPU.
* Это помогает оптимизировать размер блоков, количество потоков и уменьшить накладные расходы.

